In [5]:
!pip install pandas
!pip install nltk
!pip install fasttext-wheel

In [6]:
#import dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import re
import string
from nltk.tokenize import RegexpTokenizer # regexp tokenizers use to split words from text
from nltk.stem.snowball import SnowballStemmer # stemmes words
import tensorflow as tf

In [2]:
import tensorflow as tf

In [3]:
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
print(tf.test.is_built_with_cuda())

True


In [10]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only allocate memory as needed
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("GPU(s) is/are being used.")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("No GPU found. Using CPU.")

GPU(s) is/are being used.


In [15]:
import gc

In [16]:
import tensorflow as tf

tf.keras.backend.clear_session()

In [17]:
import gc

gc.collect()

0

In [23]:
import fasttext


In [24]:
data_fake = pd.read_csv('ISOTvfake news/Fake.csv')
data_true = pd.read_csv('ISOTvfake news/True.csv')

In [25]:
data_fake["class"] = 0
data_true["class"] = 1

In [26]:
data_fake.shape, data_true.shape

((23481, 5), (21417, 5))

In [27]:
data_merge = pd.concat([data_fake, data_true], axis = 0)
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [28]:
data = data_merge.drop(['title', 'subject', 'date'], axis = 1)

In [29]:
data.isnull().sum()


text     0
class    0
dtype: int64

In [30]:
data = data.sample(frac = 1)

data.reset_index(inplace = True)
data.drop(['index'], axis = 1, inplace = True)

In [31]:
data.head()

,text,class
0,DAKAR (Reuters) - Suspected Christian militias...,1
1,BAGHDAD (Reuters) - Iraqi lawmakers have reque...,1
2,Donald Trump s attorney Jay Sekulow made a foo...,0
3,"On more than one occasion, Condoleezza Rice wa...",0
4,ABUJA (Reuters) - Nigerian President Muhammadu...,1


In [32]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]','', text) #remove text enclosed in square brackets, including the brackets themselves
    text = re.sub("\\W", " ", text) #replaces all non-word characters (e.g., punctuation, special characters, symbols) with spaces
    text = re.sub('https?://\S+|www\.\S+', '', text) # removes URLs from the text by matching and removing both HTTP/HTTPS URLs and "www" URLs
    text = re.sub('<.*?>+', '', text) #remove HTML tags and their contents
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text) #remove all punctuation characters
    text = re.sub('\n', '', text) #removes newline characters, which are typically used to represent line breaks or paragraphs in text
    text = re.sub('\w*\d\w', '', text) #removes words containing numbers or alphanumeric patterns
    return text

In [33]:
data['text'] = data['text'].apply(wordopt)

In [34]:
tokenizer = RegexpTokenizer(r'[A-Za-z]+')#to getting alpha only
stemmer = SnowballStemmer("english")
data['text'] = data.text.map(lambda t: tokenizer.tokenize(t))
data['text'] = data.text.map(lambda l: [stemmer.stem(word) for word in l])
data['text'] = data.text.map(lambda l: ' '.join(l))

In [35]:
data['class_label'] = "__label__" + data['class'].astype(str)
data.head()

,text,class,class_label
0,dakar reuter suspect christian militia kill an...,1,__label__1
1,baghdad reuter iraqi lawmak have request that ...,1,__label__1
2,donald trump s attorney jay sekulow made a foo...,0,__label__0
3,on more than one occas condoleezza rice was ab...,0,__label__0
4,abuja reuter nigerian presid muhammadu buhari ...,1,__label__1


In [36]:
data['class_text'] = data['class_label'] + " " + data['text']
data.head()

,text,class,class_label,class_text
0,dakar reuter suspect christian militia kill an...,1,__label__1,__label__1 dakar reuter suspect christian mili...
1,baghdad reuter iraqi lawmak have request that ...,1,__label__1,__label__1 baghdad reuter iraqi lawmak have re...
2,donald trump s attorney jay sekulow made a foo...,0,__label__0,__label__0 donald trump s attorney jay sekulow...
3,on more than one occas condoleezza rice was ab...,0,__label__0,__label__0 on more than one occas condoleezza ...
4,abuja reuter nigerian presid muhammadu buhari ...,1,__label__1,__label__1 abuja reuter nigerian presid muhamm...


In [37]:
random_seed = 42

In [38]:
train, test = train_test_split(data, test_size = 0.25, random_state=random_seed)

In [39]:
x_train, x_test, y_train, y_test = train_test_split(data['text'], data['class'], test_size=0.25, random_state=random_seed)

In [40]:
train.to_csv("news.train", columns=["class_text"], index=False, header=False)
test.to_csv("news.test", columns=["class_text"], index=False, header=False)

In [41]:
# model = fasttext.train_supervised(input="news.train")
# model.test("news.test")
model = fasttext.train_supervised(input="news.train")
model.test("news.test")

(11225, 0.9906458797327394, 0.9906458797327394)

In [42]:
model.save_model("news_model.bin")

In [43]:
model = fasttext.load_model('news_model.bin')

In [44]:
model.predict('Donald Trump spent a good portion of his day at his golf club, marking the 84th day he s done so since taking the oath of office. It must have been a bad game because just after that, Trump lashed out at FBI Deputy Director Andrew McCabe on Twitter following a report saying McCabe plans to retire in a few months. The report follows McCabe s testimony in front of congressional committees this week, as well as mounting criticism from Republicans regarding the Russia probe.So, naturally, Trump attacked McCabe with a lie. How can FBI Deputy Director Andrew McCabe, the man in charge, along with leakin  James Comey, of the Phony Hillary Clinton investigation (including her 33,000 illegally deleted emails) be given $700,000 for wife s campaign by Clinton Puppets during investigation?  Trump tweeted.How can FBI Deputy Director Andrew McCabe, the man in charge, along with leakin  James Comey, of the Phony Hillary Clinton investigation (including her 33,000 illegally deleted emails) be given $700,000 for wife s campaign by Clinton Puppets during investigation?  Donald J. Trump (@realDonaldTrump) December 23, 2017He didn t stop there.FBI Deputy Director Andrew McCabe is racing the clock to retire with full benefits. 90 days to go?!!!  Donald J. Trump (@realDonaldTrump) December 23, 2017Wow,  FBI lawyer James Baker reassigned,  according to @FoxNews.  Donald J. Trump (@realDonaldTrump) December 23, 2017With all of the Intel at Trump s disposal, he s getting his information from Fox News. McCabe spent most of his career in the fight against terrorism and now he s being attacked by the so-called president. Trump has been fact-checked before on his claim of his wife receiving $700,000 for her campaign.Politifact noted in late July that Trump s  tweet about Andrew McCabe is a significant distortion of the facts. And the implication that McCabe got Clinton off as a political favor doesn t make much sense when we look at the evidence. His July tweet was rated  mostly false.  But Trump repeats these lies because he knows his supporters will believe them without bothering to Google. It s still a lie, though.Photo by Zach Gibson   Pool/Getty Images.')

(('__label__0',), array([0.99992776]))

In [46]:
# Extract vocabulary and embeddings
vocabulary = model.get_words()
word_embeddings = np.array([model.get_word_vector(word) for word in vocabulary])

In [47]:
# Create the embedding matrix
embedding_dim = model.get_dimension()  # Dimension of FastText embeddings
embedding_matrix = np.zeros((len(vocabulary), embedding_dim))
for i, word in enumerate(vocabulary):
    embedding_matrix[i] = model.get_word_vector(word)

In [48]:
# Save the embedding matrix to a file
np.save('embedding_matrix.npy', embedding_matrix)  #file is in the NumPy array format as .npy

In [49]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.models import Sequential
from keras.layers import Dropout, Flatten


In [50]:
text_column = data['text']

# Concatenate all the text data into a single string
all_text = ' '.join(text_column)

# Split the text into words and count them
word_count = len(all_text.split())
print(word_count)

18406124


In [51]:
max_sequence_length = 0
for text in text_column:
    tokens = tokenizer.tokenize(text)
    sequence_length = len(tokens)
    max_sequence_length = max(max_sequence_length, sequence_length)

In [52]:
print(max_sequence_length)

8280


In [53]:
vocab_size = len(vocabulary)
sequence_length = max_sequence_length

In [54]:
embedding_matrix.shape

(74818, 100)

In [55]:
print(type(x_train))

<class 'pandas.core.series.Series'>


In [56]:
# Convert Pandas Series to NumPy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [57]:
# Check the data types and shapes to verify they are suitable
print("x_train type:", type(x_train))
print("x_train shape:", x_train.shape)
print("y_train type:", type(y_train))
print("y_train shape:", y_train.shape)
print("x_test type:", type(x_test))
print("x_test shape:", x_test.shape)
print("y_test type:", type(y_test))
print("y_test shape:", y_test.shape)

x_train type: <class 'numpy.ndarray'>
x_train shape: (33673,)
y_train type: <class 'numpy.ndarray'>
y_train shape: (33673,)
x_test type: <class 'numpy.ndarray'>
x_test shape: (11225,)
y_test type: <class 'numpy.ndarray'>
y_test shape: (11225,)


In [58]:
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [59]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)  # Assuming x_train is a list of text strings
x_train_sequences = tokenizer.texts_to_sequences(x_train)
x_test_sequences = tokenizer.texts_to_sequences(x_test)

# Pad sequences
x_train_padded = pad_sequences(x_train_sequences, maxlen=sequence_length)
x_test_padded = pad_sequences(x_test_sequences, maxlen=sequence_length)

In [60]:
from keras.models import Model
from keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Concatenate, GlobalMaxPooling1D, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [61]:
# Input layer
input_layer = Input(shape=(sequence_length,))

# Embedding layer
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], trainable=False)(input_layer)

# Convolutional layers
conv1d = Conv1D(filters=128, kernel_size=3, activation='relu')(embedding)
conv1d_1 = Conv1D(filters=128, kernel_size=4, activation='relu')(embedding)
conv1d_2 = Conv1D(filters=128, kernel_size=5, activation='relu')(embedding)

# MaxPooling layers
max_pooling1d = MaxPooling1D(pool_size=2)(conv1d)
max_pooling1d_1 = MaxPooling1D(pool_size=2)(conv1d_1)
max_pooling1d_2 = MaxPooling1D(pool_size=2)(conv1d_2)

# Concatenate max-pooled layers
concatenated = Concatenate(axis=1)([max_pooling1d, max_pooling1d_1, max_pooling1d_2])

# Additional Convolutional and MaxPooling layers
conv1d_3 = Conv1D(filters=128, kernel_size=3, activation='relu')(concatenated)
max_pooling1d_3 = MaxPooling1D(pool_size=2)(conv1d_3)
conv1d_4 = Conv1D(filters=128, kernel_size=3, activation='relu')(max_pooling1d_3)
max_pooling1d_4 = MaxPooling1D(pool_size=2)(conv1d_4)

# GlobalMaxPooling layer
global_max_pooling1d = GlobalMaxPooling1D()(max_pooling1d_4)

# Dense layers
dense = Dense(128, activation='relu')(global_max_pooling1d)
output_layer = Dense(1, activation='sigmoid')(dense)  # Assuming binary classification

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model (specify loss, optimizer, and metrics)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Define model checkpoint
model_checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Print a summary of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 8280)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 8280, 100)    7481800     ['input_1[0][0]']                
                                                                                                  
 conv1d (Conv1D)                (None, 8278, 128)    38528       ['embedding[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 8277, 128)    51328       ['embedding[0][0]']              
                                                                                              

In [65]:
model.fit(x_train_padded, y_train, validation_data=(x_test_padded, y_test), epochs=5, batch_size=20, callbacks=[early_stopping, model_checkpoint])

Epoch 1/5
1684/1684 [==============================] - 530s 311ms/step - loss: 0.0130 - accuracy: 0.9968 - val_loss: 0.0187 - val_accuracy: 0.9952
Epoch 2/5
1684/1684 [==============================] - 516s 306ms/step - loss: 0.0136 - accuracy: 0.9967 - val_loss: 0.0133 - val_accuracy: 0.9963
Epoch 3/5
1684/1684 [==============================] - 514s 305ms/step - loss: 0.0126 - accuracy: 0.9974 - val_loss: 0.0153 - val_accuracy: 0.9960
Epoch 4/5
1684/1684 [==============================] - 521s 309ms/step - loss: 0.0128 - accuracy: 0.9969 - val_loss: 0.0096 - val_accuracy: 0.9981
Epoch 5/5
1684/1684 [==============================] - 557s 331ms/step - loss: 0.0102 - accuracy: 0.9980 - val_loss: 0.0111 - val_accuracy: 0.9975


In [66]:
loss, accuracy = model.evaluate(x_test_padded, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

351/351 [==============================] - 51s 144ms/step - loss: 0.0111 - accuracy: 0.9975
Test Loss: 0.0111327413469553, Test Accuracy: 0.9975055456161499


In [67]:
model.save('models/fasttext-cnn-model.h5')

In [73]:
tokenizer = RegexpTokenizer(r'[A-Za-z]+') #to getting alpha only
stemmer = SnowballStemmer("english")

# Tokenize and preprocess the new input data
new_input_data = [ "BREAKING : Trump Expressed Concern Over Anthony Weinerâ€™s â€œIllegal Accessâ€ to Classified Info 2 Months ago BREAKING : Trump Expressed Concern Over Anthony Weinerâ€™s â€œIllegal Accessâ€ to Classified Info 2 Months ago Breaking News By Amy Moreno October 28, 2016. Once again, Trump was right. Back in August, in a statement regarding Hillaryâ€™s carelessness handling classified documents, Trump stated that he was concerned that Weiner had â€œaccessâ€ to information he shouldnâ€™t. Now that weâ€™re learning that the FBI discovered â€œnew emailsâ€ on a â€œdeviceâ€ associated to Weiner, it looks as if Trump was right AGAIN. â€” Deplorable AJ (@asamjulian) October 28, 2016 This is a movement â€“ we are the political OUTSIDERS fighting against the FAILED GLOBAL ESTABLISHMENT! Join the resistance and help us fight to put America First! Amy Moreno is a Published Author , Pug Lover & Game of Thrones Nerd. You can follow her on Twitter here and Facebook here . Support the Trump Movement and help us fight Liberal Media Bias. Please LIKE and SHARE this story on Facebook or Twitter.  "]
new_sequences = [wordopt(sentence) for sentence in new_input_data]

from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(new_sequences)  # Assuming x_train is a list of text strings
new_sequences = tokenizer1.texts_to_sequences(new_sequences)

# Pad sequences
new_sequences = pad_sequences(new_sequences, maxlen=sequence_length)

# new_sequences = [map_tokens_to_indices(sequence, embedding_matrix, sequence_length) for sequence in new_sequences]

# Convert 'new_sequences' to a NumPy array
new_data = np.array(new_sequences)

# Pad or truncate the sequences to match the sequence length
new_data = pad_sequences(new_data, maxlen=sequence_length, padding='post', truncating='post')

# Make predictions on the new data
predictions = model.predict(new_data)

# The 'predictions' array will contain probability scores for each class (0 and 1)
# You can convert these scores to class labels based on a threshold (e.g., 0.5)
predicted_labels = [1 if score >= 0.5 else 0 for score in predictions]

class_mapping = {0: "fake", 1: "true"}

# Use the mapping to transform the predicted labels
predicted_class_names = [class_mapping[label] for label in predicted_labels]

print(predicted_class_names)


1/1 [==============================] - 0s 391ms/step
['fake']


In [71]:
from tensorflow.keras.models import load_model

# Load your trained CNN model
model1 = load_model('models/fasttext-cnn-model.h5')

In [74]:
# tokenizer = RegexpTokenizer(r'[A-Za-z]+') #to getting alpha only
# stemmer = SnowballStemmer("english")
from tensorflow.keras.preprocessing.text import Tokenizer

# Tokenize and preprocess the new input data
new_input_data = ["The fallout from Ryan Lochteâ€™s story about being robbed at gunpoint in Rio  â€”   a tale the Brazilian police said was not true  â€”   continued Monday when four companies said they would end business partnerships with Mr. Lochte, an American swimmer and   Olympic medalist. After a week of intense international media attention and anger in Brazil, the financial repercussions were swift for Mr. Lochte as Speedo USA, the luxury clothing retailer Ralph Lauren and the mattress company Airweave all announced that they would part ways with him. And Syneron Candela, a company that sells   devices, told Reuters its relationship with the swimmer ended on Sunday. Speedo USA said in a message on Twitter that it would instead donate a $50, 000 portion of Mr. Lochteâ€™s fee to a charity to help Brazilian children. â€œWhile we have enjoyed a winning relationship with Ryan for over a decade and he has been an important member of the Speedo team, we cannot condone behavior that is counter to the values this brand has long stood for,â€ the company said in its statement. On Monday, Kim Angelastro, a spokeswoman for Syneron Candela, wrote in an email, â€œWe hold our employees to high standards, and we expect the same of our business partners. â€ Mr. Lochte was a spokesman for the companyâ€™s Gentle Hair Removal brand. Through a spokeswoman, Ralph Lauren said Monday that Mr. Lochteâ€™s endorsement agreement with the clothing company had been for only the 2016 Olympics, and that his contract would not be renewed. Airweave said on Twitter that â€œafter careful consideration, we have made the decision to end our partnership with Ryan Lochte. â€ The decisions to cut ties with Mr. Lochte, 32, were the first major signs of the financial fallout for him. For the past week, he has been at the center of an international firestorm after the Brazilian police said he and three other American athletes  â€”   Jimmy Feigen, Jack Conger and Gunnar Bentz  â€”   had fabricated the account of being robbed after a   party in Rio de Janeiro. The authorities said that the swimmers had instead drunkenly vandalized a gas station bathroom, paying a security guard about $50 for the damage before leaving. Mr. Lochte originally said that the car they were traveling in had been stopped by armed men, who held a gun to his head. But his story later changed. A Brazilian judge ordered the swimmers to remain in Rio, but Mr. Lochte had already left the country. After Mr. Conger and Mr. Bentz were pulled from their flight to the United States, they told the police that the confrontation began when Lochte pulled a poster off a gas station wall. Mr. Feigen, 26, later donated $10, 800 to a charity in Rio that teaches martial arts to poor children. Mr. Lochte first issued an apology on social media  â€”   â€œI should have been much more responsible for how I handled myself,â€ he wrote  â€”   then told Matt Lauer in an interview on NBC that he had been intoxicated and that he had â€œoverexaggerated that story. â€ He has maintained that he was held at gunpoint. â€œAll we know is that there was a gun pointed in our direction, and we were demanded to give money,â€ Mr. Lochte said. Mr. Lochte, whose boyish and sometimes oafish personality had made him a commercial success in Olympics past, had headed into Rio with fewer sponsors than heâ€™d had at the London Games, according to a report by CNN Money. Mr. Lochte took home a gold medal in the   freestyle relay in Rio.", "Republicans are working overtime trying to sell their scam of a tax bill to the public as something that directly targets middle-class and working-class families with financial relief. Nothing could be further from the truth, and they re getting hammered on that repeatedly. Speaking on CNBC, Paul Ryan was going full throttle, trying to convince us that the paltry savings we re getting is actually wait for it big money.But he didn t just go with the usual talking points. With a smug look that only someone who grew up in a wealthy family can muster when talking about that which he does not know, Ryan claimed that the $2,059 more per year that families living paycheck-to-paycheck will see is extremely significant. Then he decided he had to amend that to say such savings might be nothing to a family earning $600,000 per year (true), or for people living in New York or California (false).Those are the same two states that Trump s loyal subjects insist on stripping from the 2016 vote totals to claim that Trump actually won the popular vote. Watch Ryan completely dismiss all the struggling families living in blue states below:If you re living paycheck-to-paycheck which is more than half of the people in this country and you got #2059more from a tax cut next year, that s not nothing. pic.twitter.com/8TKtrMqRa1  Paul Ryan (@SpeakerRyan) December 21, 2017Someone needs to reach through their computer or television and wipe that smugness off his face. It is the height of arrogance and insult to imply that there are no struggling families in either of those two states.Featured image via Mark Wilson/Getty Images","BREAKING : Trump Expressed Concern Over Anthony Weinerâ€™s â€œIllegal Accessâ€ to Classified Info 2 Months ago BREAKING : Trump Expressed Concern Over Anthony Weinerâ€™s â€œIllegal Accessâ€ to Classified Info 2 Months ago Breaking News By Amy Moreno October 28, 2016. Once again, Trump was right. Back in August, in a statement regarding Hillaryâ€™s carelessness handling classified documents, Trump stated that he was concerned that Weiner had â€œaccessâ€ to information he shouldnâ€™t. Now that weâ€™re learning that the FBI discovered â€œnew emailsâ€ on a â€œdeviceâ€ associated to Weiner, it looks as if Trump was right AGAIN. â€” Deplorable AJ (@asamjulian) October 28, 2016 This is a movement â€“ we are the political OUTSIDERS fighting against the FAILED GLOBAL ESTABLISHMENT! Join the resistance and help us fight to put America First! Amy Moreno is a Published Author , Pug Lover & Game of Thrones Nerd. You can follow her on Twitter here and Facebook here . Support the Trump Movement and help us fight Liberal Media Bias. Please LIKE and SHARE this story on Facebook or Twitter.  "]
new_sequences = [wordopt(sentence) for sentence in new_input_data]

from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(new_sequences)  # Assuming x_train is a list of text strings
new_sequences = tokenizer1.texts_to_sequences(new_sequences)

# Pad sequences
new_sequences = pad_sequences(new_sequences, maxlen=sequence_length)

# new_sequences = [map_tokens_to_indices(sequence, embedding_matrix, sequence_length) for sequence in new_sequences]

# Convert 'new_sequences' to a NumPy array
new_data = np.array(new_sequences)

# Pad or truncate the sequences to match the sequence length
new_data = pad_sequences(new_data, maxlen=sequence_length, padding='post', truncating='post')

# Make predictions on the new data
predictions = model1.predict(new_data)

# The 'predictions' array will contain probability scores for each class (0 and 1)
# You can convert these scores to class labels based on a threshold (e.g., 0.5)
predicted_labels = [1 if score >= 0.5 else 0 for score in predictions]

class_mapping = {0: "fake", 1: "true"}

# Use the mapping to transform the predicted labels
predicted_class_names = [class_mapping[label] for label in predicted_labels]

print(predicted_class_names)


1/1 [==============================] - 0s 227ms/step
['fake', 'fake', 'true']
